<a href="https://colab.research.google.com/github/Adrok24/Proyecto-Integrador/blob/branch_1/Preprocessing_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
# Imports
from google.colab import files
from google.colab import drive
import os
from os.path import isfile, join
import pandas as pd
import re

In [ ]:
drive.mount('/content/drive')

In [ ]:
folder = '/content/drive/My Drive/Colab Notebooks/desafio final/'

path_to_names = os.path.join(folder, 'list_names.csv')
data = pd.read_csv(path_to_names)

In [ ]:
list_names = data.Name.tolist()

In [2]:
!unrar x '/content/drive/My Drive/Colab Notebooks/desafio final/txt_books_spa.rar'


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/My Drive/Colab Notebooks/desafio final/txt_books_spa.rar

Creating    txt_books_spa                                             OK
Creating    txt_books_spa/cortazar txt                                OK
Extracting  txt_books_spa/cortazar txt/alvo_el_crepusculo_-_Julio_Cortazar.txt       1%  OK 
Extracting  txt_books_spa/cortazar txt/ases_de_literatura_-_Julio_Cortazar.txt       3%  OK 
Extracting  txt_books_spa/cortazar txt/Bestiario_-_Julio_Cortazar.txt       5%  OK 
Extracting  txt_books_spa/cortazar txt/Cartas_1937-1954_-_Julio_Cortazar.txt      11%  OK 
Extracting  txt_books_spa/cortazar txt/Cartas_1955-1964_-_Julio_Cortazar.txt      18%  OK 
Extracting  txt_books_spa/cortazar txt/Deshoras_-_Julio_Cortazar.txt      20%  OK 
Extracting  txt_books_spa/cortazar txt/Juegos-Julio-Cortazar.txt          21%  OK 
Extracting  txt

In [31]:
mypath = '/content/txt_books_spa' 

# list_files = [join(root, name_file) for root, dirs, name_file in os.walk(mypath, topdown=False)] 

count_files = 0
dict_paths = {}
for root, dirs, files in os.walk(mypath, topdown=False):
    for name_file in files:
        dict_paths[count_files] = {"path": os.path.join(root, name_file), "name": name_file.replace('.txt', ''), "genre": root.split('/')[3].split(' ')[0], "category" : "Book"}
        count_files = count_files + 1

df_path = pd.DataFrame.from_dict(dict_paths, "index")
display(df_path.shape)

(84, 4)

In [33]:
df_path.columns

Index(['path', 'name', 'genre', 'category'], dtype='object')

In [36]:
!pip install nltk

In [37]:
 from nltk.tokenize import sent_tokenize, word_tokenize

In [38]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [104]:
def get_lines(line, book, dict_book_final, count_lines):
  if (len(line) > 170):
    split_line = line.split(',')
    for s_line in split_line:
      if ((len(s_line) < 80) and (len(s_line) > 30)):
        dict_book_final[count_lines] = {"line": s_line, "len": len(s_line), "book": book['name'], "genre": book['genre'], "category": book['category']}
        count_lines = count_lines + 1
  elif (len(line) > 30):  
    dict_book_final[count_lines] = {"line": line, "len": len(line), "book": book['name'], "genre": book['genre'], "category": book['category']}
    count_lines = count_lines + 1
  return dict_book_final, count_lines

In [105]:
dict_book_final = {}
count_lines = 0
for index, book in df_path.iterrows():
  corpus = open(book['path'], "r")
  cadena = corpus.read()
  list_cadena = sent_tokenize(cadena, language='spanish')
  print('cant sent:',len(list_cadena),book['name'])

  for line in list_cadena:
    if (line.find('\n') == -1):
      dict_book_final, count_lines = get_lines(line, book, dict_book_final, count_lines)
    else:
      split_line = line.split('\n')
      for s_line in split_line:
        dict_book_final, count_lines = get_lines(s_line, book, dict_book_final, count_lines)

cant sent: 544 _versos_perdidos_-_Txus_di_Fellatio
cant sent: 283 _medianamente_dulces_-_Alma_Phillip
cant sent: 325 ntologia_poetica_-_Antonio_Gamoneda
cant sent: 162 Aquella_noche_-_Cristina_Peri_Rossi
cant sent: 350 Susurros_del_Alma_-_Alec_Blackhell
cant sent: 60 Sombra_-_Leopoldo_Maria_Panero
cant sent: 805 o_de_una_chica_en_llamas_-_SrtaBebi
cant sent: 828 _al_viento_Poemas_-_Diego_Fortunato
cant sent: 2039 Poemas_-_Fernando_Pessoa
cant sent: 330 as_cuatro_estaciones_-_Basho_Matsuo
cant sent: 35 izofrenicos_-_Leopoldo_Maria_Panero
cant sent: 377 _grandes_mujeres_-_Alfonsina_Storni
cant sent: 274 uaderno_del_delirio_-_Elvira_Daudet
cant sent: 727 aricias_al_tiempo_-_Diego_Fortunato
cant sent: 1076 Amor_y_asco_-_SrtaBebi
cant sent: 2346 _cosas_que_escribi_-_Ainhoa_Escarti
cant sent: 18 onversacion_-_Leopoldo_Maria_Panero
cant sent: 64 e_la_locura_-_Leopoldo_Maria_Panero
cant sent: 226 ado_de_exilio_-_Cristina_Peri_Rossi
cant sent: 2259 Vetas_profundas_-_Fernando_Aramburu
cant sent:

In [106]:
df_book_lines = pd.DataFrame.from_dict(dict_book_final, "index")
display(df_book_lines)

,line,len,book,genre,category
0,EL CEMENTERIO DE LOS VERSOS PERDIDOS,36,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
1,obra del Príncipe de la Dulce Pena,35,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
2,más conocido por todos como Txus di Fellatio,45,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
3,y un disco de diamante por la venta de un mil...,63,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
4,son el testimonio de la historia viva que est...,67,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
...,...,...,...,...,...
181355,"Madrid: Editorial Losada, 2006.",31,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book
181356,"Barcelona: Editorial Kairós, 2003.",34,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book
181357,"TOLA, Fernando; DRAGONETTI, Carmen.",35,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book
181358,Le mythe de la liberté et la voie de la médita...,51,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book


In [114]:
def preprocess_sentence(sentence):
  sentence = sentence.strip()
  # vamos a crear espacios entre las frases y los puntos:
  # ej: "he is a boy." => "he is a boy ."
  sentence = re.sub(r"([?.!,:])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)
  sentence = re.sub(r'[A-Z]{2,20}', " ", sentence)
  sentence = sentence.strip()
  return sentence

In [115]:
df_book_lines.iloc[:,0] = df_book_lines.iloc[:,0].apply(preprocess_sentence)

In [116]:
df_book_lines

,line,len,book,genre,category
0,,36,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
1,obra del Príncipe de la Dulce Pena,35,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
2,más conocido por todos como Txus di Fellatio,45,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
3,y un disco de diamante por la venta de un mill...,63,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
4,son el testimonio de la historia viva que está...,67,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
...,...,...,...,...,...
181355,"Madrid : Editorial Losada , 2006 .",31,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book
181356,"Barcelona : Editorial Kairós , 2003 .",34,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book
181357,", Fernando; , Carmen .",35,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book
181358,Le mythe de la liberté et la voie de la médita...,51,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book


In [119]:
from IPython.display import clear_output
list_new_rows = [] 
for index, row in df_book_lines.iterrows():
  new_row = row['line']
  for word in row['line'].split(' '):
    if word in list_names:
      new_row = row['line'].replace(word, '@name')
    if word in [':','[',']','&','%']:
      new_row = row['line'] = '[DELETE]'

  clear_output(wait=True)
  print(index)   
  list_new_rows.append(new_row)

181359


In [120]:
df_book_lines.iloc[:,0] = list_new_rows

In [121]:
df_book_lines

,line,len,book,genre,category
0,,36,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
1,obra del Príncipe de la Dulce @name,35,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
2,más conocido por todos como Txus di Fellatio,45,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
3,y un disco de diamante por la venta de un mill...,63,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
4,son el testimonio de la historia viva que está...,67,_versos_perdidos_-_Txus_di_Fellatio,poesía,Book
...,...,...,...,...,...
181355,[DELETE],31,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book
181356,[DELETE],34,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book
181357,", Fernando; , Carmen .",35,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book
181358,@name mythe de la liberté et la voie de la méd...,51,l_budismo_-_Ernest_Yassine_Bendriss,filosofia,Book


In [142]:
borrar = df_book_lines.line.value_counts().head(6).index.tolist()
borrar

['[DELETE]',
 '',
 'Versión de @name Martínez Montávez',
 '@name Canciones de Mihyar el de Damasco 1961',
 'A    È  Y  ÍA',
 'Y']

In [153]:
mask_del = df_book_lines.line.apply(lambda x: True if x in borrar else False) 
df_book_lines[mask_del]

Int64Index([     0,     11,     12,     13,     19,     37,     40,     67,
               123,    142,
            ...
            181343, 181344, 181346, 181347, 181349, 181351, 181354, 181355,
            181356, 181359],
           dtype='int64', length=9830)

In [155]:
df_book_lines.drop(df_book_lines[mask].index, inplace=True)

In [156]:
df_book_lines.drop_duplicates(inplace=True)
df_book_lines.shape[0]

170981

In [160]:
df_book_lines.len = df_book_lines.line.apply(lambda x : len(x))

In [165]:
mask_largo = df_book_lines.len < 35
print(mask_largo.sum())

15802


In [166]:
df_book_lines.drop(df_book_lines[mask_largo].index, inplace=True)
df_book_lines.shape[0]

155179

In [168]:
df_book_lines.sample(15)

,line,len,book,genre,category
156067,Tomemos como ejemplo la inmoralidad erótica,43,ogia_del_sofista_-_Fernando_Savater,filosofia,Book
58951,pero eso no impedirá en absoluto que nos veamo...,75,Cartas_1955-1964_-_Julio_Cortazar,cortazar,Book
66756,la silla y él mismo asumían proporciones inqui...,55,Los-premios-Julio-Cortazar,cortazar,Book
91073,lo que luego se sabría a la luz de la autocrít...,62,Obra_critica_III_-_Julio_Cortazar,cortazar,Book
110646,@name y yo entramos en la Unesco hace dos días...,86,Cartas_1937-1954_-_Julio_Cortazar,cortazar,Book
100782,"Tonto de mí , que pensé líricamente en la posi...",114,Cartas_1937-1954_-_Julio_Cortazar,cortazar,Book
174521,A continuación se emprende el estudio del ente...,84,Acerca_del_alma_-_Aristoteles,filosofia,Book
5407,Minha idéia de ti é um cadáver que o mar traz ...,71,Poemas_-_Fernando_Pessoa,poesía,Book
90918,@name margen de la circunstancia local,38,Obra_critica_III_-_Julio_Cortazar,cortazar,Book
67371,—¿Usted se va a hacer cortar el pelo ?,38,Los-premios-Julio-Cortazar,cortazar,Book


In [169]:
df_book_lines.to_csv('/content/drive/My Drive/Colab Notebooks/desafio final/train_corpus_v2.csv')